### Task 1


In [26]:
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import from_numpy
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.nn import BCELoss

from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

from torch.optim import SGD


class MyDataset(Dataset):

    def __init__(self, path):
        df = read_csv(path, header=None)

        self.X = df.values[:, :-1]
        self.X = self.X.astype('float32')
        self.X = from_numpy(self.X)

        self.y = df.values[:, -1]
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
        self.y = from_numpy(self.y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def get_splits(self, test_size=0.33):
        test_size = round(test_size * len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])


class RADAR_MLP(Module):
    def __init__(self, n_inputs):
        super(RADAR_MLP, self).__init__()

        self.hidden1 = Linear(n_inputs, 16)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()

        self.hidden2 = Linear(16, 10)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()

        self.hidden3 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act3 = ReLU()

        self.hidden4 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act4 = Sigmoid()

    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)
        X = self.hidden4(X)
        X = self.act4(X)
        return X


def get_data_loaders(path):
    dataset = MyDataset(path)
    train, test = dataset.get_splits()
    train_dl = DataLoader(train, batch_size=16, shuffle=True)
    test_dl = DataLoader(test, batch_size=16, shuffle=True)
    return train_dl, test_dl


def train_BCE_model(train_dl, model):
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.1, momentum=0.9)

    for epoch in range(11):
        for i, (inputs, targets) in enumerate(train_dl):
            optimizer.zero_grad()
            yhat = model(inputs)
            loss = criterion(yhat, targets)
            loss.backward()
            optimizer.step()


def get_acc(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        yhat = model(inputs)
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))

        yhat = yhat.round()
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)

    acc = accuracy_score(actuals, predictions)
    return acc


if __name__ == '__main__':
    path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
    train_dl, test_dl = get_data_loaders(path)
    model = RADAR_MLP(34)
    train_BCE_model(train_dl, model)
    acc = get_acc(test_dl, model)
    print('Accuracy: %.3f' % acc)



Accuracy: 0.862


### Task 2

In [33]:
from numpy import sqrt
from torch.nn import MSELoss
from sklearn.metrics import mean_squared_error

class HOUSING_MLP(Module):
    def __init__(self, n_inputs):
        super(HOUSING_MLP, self).__init__()

        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()

        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()

        self.hidden3 = Linear(8, 6)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act3 = ReLU()

        self.hidden4 = Linear(6, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act4 = Sigmoid()

    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)
        X = self.hidden4(X)
        X = self.act4(X)
        return X

 
def train_MSE_model(train_dl, model):
    criterion = MSELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    for epoch in range(11):
        for i, (inputs, targets) in enumerate(train_dl):
            optimizer.zero_grad()
            yhat = model(inputs)
            loss = criterion(yhat, targets)
            loss.backward()
            optimizer.step()

def get_mse(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        yhat = model(inputs)
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))

        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    mse = mean_squared_error(actuals, predictions)
    return mse
 
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
train_dl, test_dl = get_data_loaders(path)
model = HOUSING_MLP(13)
train_MSE_model(train_dl, model)
mse = get_mse(test_dl, model)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))

MSE: 14670.443, RMSE: 121.122
